# Data Forecasting

The module is in charge of: 
- forecasting the value of temperature in the next X seconds; 
- forecasting the value of humidity in the next X seconds; 
- forecasting the value of the gas sensor in the next X seconds 

(X is defined by users or developers). 

The time-series of the predicted values must be saved on INFLUX and shown on the GRAFANA dashboard as well; e.g. by using two lines for the temperature graph, one related to raw measurements, one to predictions. One or multiple techniques for time-series forecasting can be developed and compared.


## Import Dependencies

In [ ]:
!pip3 install matplotlib
!pip3 install pandas
!pip3 install seaborn
!pip3 install influxdb_client

In [92]:
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import chart_studio.plotly as plotly
import plotly.figure_factory as ff
from plotly import graph_objs as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from datetime import datetime,timezone
import dateutil.parser
import time
from datetime import timedelta

# Data retrival from influx

In [94]:
token = 'gYkhnh7Ft18AtFX8m7BEn3RQGb4vb9tcE_AbWkXFfXgpqU69ad3DQsKPvHW959iqN-mLFZW2wi6Pc7-XmohEAA=='
bucket = 'temperature'
org = 'iot_group'
client = InfluxDBClient(url="http://192.168.178.39:8086", token=token, org=org)
query_api = client.query_api()
write_api = client.write_api(write_options=SYNCHRONOUS)

In [95]:
query = 'from(bucket: "temperature")' \
  '|> range(start:2022-08-10T15:00:00Z, stop:2022-09-20T15:00:00Z)'

In [ ]:
result = client.query_api().query(org=org, query=query)
print(result)

-Exploiting the query results in order to obtain a dataframe

## Implementation Temperature Predictor

In [133]:
import pandas as pd
import numpy as np
import chart_studio.plotly as plotly
import plotly.figure_factory as ff
from plotly import graph_objs as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from datetime import datetime,timezone
import dateutil.parser
import time
from datetime import timedelta

In [ ]:
!pip3 install chart_studio
!pip3 install fbprophet

In [ ]:
dataset_path = "/Users/riccardobaratin/Documents/GitHub/iot-project/datasets/dataset_temp_bolo.csv"
df = pd.read_csv(dataset_path)
df.head()
#for t in df['_time'].values:
#     d=dateutil.parser.parse(t)
#     aux=d.strftime('%Y-%m-%d')
#     df['_time'].replace({t:aux },inplace=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["_time"], y=df['_value'], name='temperature',line_color='green'))
fig.layout.update(title_text='Time Series data with Rangeslider',xaxis_rangeslider_visible=True)
fig

In [ ]:
X = df[["_time", "_value"]]
y = df.iloc[:,5]
print(y)

In [ ]:
train_df = pd.DataFrame()
train_df["ds"] = pd.to_datetime(X["_time"])
train_df["y"]=y
train_df.head(2)

In [ ]:
model = Prophet()
model.fit(train_df)
future = model.make_future_dataframe(periods=365)
future.tail(2)

In [ ]:
forecast = model.predict(future)
fig1 = plot_plotly(model, forecast)
fig1

In [ ]:
#plot component wise forecast
fig2 = model.plot_components(forecast)